<a href="https://colab.research.google.com/github/amandarcrangel/engenharia-analytics-fiap/blob/main/An%C3%A1lise_missings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Análise de Missings

Bibliotecas

In [1]:
# gerais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Dados

In [2]:
# carregar dados
dados_hatco = pd.read_csv('hatco_csv.csv', sep=';', decimal=',')

In [3]:
# dados hatco
dados_hatco.head()

,id,velocidade_entrega,nivel_preco,flexibilidade_preco,imagem_fornecedor,qualidade_servico,imagem_forca_vendas,qualidade_produto,tamanho_firma,nivel_uso,nivel_satisfacao,area_industria
0,1,4.1,0.6,6.9,4.7,2.4,2.3,5.2,pequena,32.0,4.2,industria
1,2,1.8,3.0,6.3,6.6,2.5,4.0,8.4,grande,43.0,4.3,outra_area
2,3,3.4,5.2,5.7,6.0,4.3,2.7,8.2,grande,48.0,5.2,industria
3,4,2.7,1.0,7.1,5.9,1.8,2.3,7.8,grande,32.0,3.9,industria
4,5,6.0,0.9,9.6,7.8,3.4,4.6,4.5,pequena,58.0,6.8,industria


Indentificação dos MISSINGS

In [10]:
# contar as linhas
missings_por_linha = pd.DataFrame({'id': dados_hatco['id'].tolist(),
                                   'n_missings': dados_hatco.isna().sum(axis=1).tolist()})
n_colunas = dados_hatco.shape[1] - 1


In [11]:
missings_por_linha

,id,n_missings
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
165,266,0
166,267,2
167,268,1
168,269,2


In [12]:
# vamos tirar -1 a coluna de ID
n_colunas = dados_hatco.shape[1] -1
n_colunas

11

In [13]:
# verificar os resultados = vamos calcular a coluna nova
missings_por_linha     .assign( perc_missings = missings_por_linha['n_missings'] / n_colunas ) \
    .sort_values('perc_missings', ascending = False) \
    .head(10)

,id,n_missings,perc_missings
162,263,7,0.636364
109,210,7,0.636364
113,214,6,0.545455
160,261,6,0.545455
144,245,5,0.454545
132,233,5,0.454545
103,204,3,0.272727
120,221,3,0.272727
123,224,3,0.272727
106,207,3,0.272727


Obter o número de missings por coluna

In [15]:
#somar todos missing por coluna
missings_por_coluna = dados_hatco.isna().sum()
missings_por_coluna

,0
id,0
velocidade_entrega,21
nivel_preco,13
flexibilidade_preco,17
imagem_fornecedor,7
qualidade_servico,9
imagem_forca_vendas,6
qualidade_produto,9
tamanho_firma,2
nivel_uso,9


In [16]:
# contar as colunas
missings_por_coluna = dados_hatco.isna().sum()
missings_por_coluna = pd.DataFrame( missings_por_coluna,
                                   columns=['n']) \
                                   .reset_index() \
                                   .rename(columns={'index': 'variaveis'})


In [17]:
missings_por_coluna

,variaveis,n
0,id,0
1,velocidade_entrega,21
2,nivel_preco,13
3,flexibilidade_preco,17
4,imagem_fornecedor,7
5,qualidade_servico,9
6,imagem_forca_vendas,6
7,qualidade_produto,9
8,tamanho_firma,2
9,nivel_uso,9


In [9]:
# verificar os resultados percentual
missings_por_coluna \
    .assign( perc_missings = missings_por_coluna['n'] / n_linhas) \
    .sort_values('n', ascending = False)

,variaveis,n,perc_missings
1,velocidade_entrega,21,0.123529
3,flexibilidade_preco,17,0.100000
2,nivel_preco,13,0.076471
5,qualidade_servico,9,0.052941
7,qualidade_produto,9,0.052941
9,nivel_uso,9,0.052941
4,imagem_fornecedor,7,0.041176
10,nivel_satisfacao,7,0.041176
6,imagem_forca_vendas,6,0.035294
8,tamanho_firma,2,0.011765


Comparar as váriavéis qualitativas

In [18]:
dados_hatco['tamanho_firma'] \
            .value_counts(dropna=False) \
            .to_frame('n') \
            .reset_index() \
            .rename(columns={'index': 'tamanho_firma'})

,tamanho_firma,n
0,pequena,103
1,grande,65
2,NaN,2


Comparando duas váriaveis *quantitativas*

In [19]:
pd.crosstab(dados_hatco.tamanho_firma.fillna('NA'),
            dados_hatco.area_industria.fillna('NA'))

area_industria,NA,industria,outra_area
tamanho_firma,,,
NA,0,0,2
grande,1,32,32
pequena,0,51,52


Comparar quantitativas

In [20]:
# filtramos os missings -eliminamos apenas sem tratar
dados_hatco[['velocidade_entrega', 'qualidade_produto']].corr()

,velocidade_entrega,qualidade_produto
velocidade_entrega,1.000000,-0.404352
qualidade_produto,-0.404352,1.000000


In [21]:
# atribuir zero = fillna
dados_hatco[['velocidade_entrega', 'qualidade_produto']].fillna(0).corr()

,velocidade_entrega,qualidade_produto
velocidade_entrega,1.00000,-0.12319
qualidade_produto,-0.12319,1.00000


Tratamento

Eliminar os missings

In [22]:
# dados totais
dados_hatco.shape

(170, 12)

In [23]:
# eliminar missings
dados_hatco.dropna().shape

(126, 12)

In [25]:
# obter o mais frequente
dados_hatco['area_industria'] \
      .value_counts(dropna=False) \
      .to_frame('n') \
      .reset_index() \
      .rename(columns={'index': 'area_industria'})


,area_industria,n
0,outra_area,86
1,industria,83
2,NaN,1


In [24]:
# imputar dados pelo mais frequente
dados_hatco['area_industria'] \
      .fillna('outra_area') \
      .value_counts(dropna=False) \
      .to_frame('n') \
      .reset_index() \
      .rename(columns={'index': 'area_industria'})

,area_industria,n
0,outra_area,87
1,industria,83


Atribuição média/mediana

In [27]:
# obter as estatisticas
stats = dados_hatco[['flexibilidade_preco', 'imagem_fornecedor']].describe()
stats

,flexibilidade_preco,imagem_fornecedor
count,153.000000,163.000000
mean,7.952288,5.217178
std,1.391378,1.144099
min,5.000000,2.500000
25%,6.700000,4.500000
50%,8.200000,5.000000
75%,9.200000,6.000000
max,10.000000,8.200000


In [28]:
# imputar os dados
dados_hatco2 = dados_hatco \
    .assign(flexibilidade_preco2 = dados_hatco['flexibilidade_preco'].fillna(stats['flexibilidade_preco'].loc['50%']) ) \
    .assign(imagem_fornecedor2 = dados_hatco['imagem_fornecedor'].fillna(stats['imagem_fornecedor'].loc['mean']) )


In [30]:
# ver os resultados
dados_hatco2[['id', 'flexibilidade_preco', 'flexibilidade_preco2', 'imagem_fornecedor', 'imagem_fornecedor2']].tail(5)

,id,flexibilidade_preco,flexibilidade_preco2,imagem_fornecedor,imagem_fornecedor2
165,266,9.4,9.4,4.9,4.900000
166,267,NaN,8.2,NaN,5.217178
167,268,9.7,9.7,3.3,3.300000
168,269,NaN,8.2,4.5,4.500000
169,270,8.7,8.7,4.6,4.600000


In [29]:
# verificar os resultados
missings_por_coluna \
    .assign( perc_missings = missings_por_coluna['n'] / n_linhas) \
    .sort_values('n', ascending = False)

,variaveis,n,perc_missings
1,velocidade_entrega,21,0.123529
3,flexibilidade_preco,17,0.100000
2,nivel_preco,13,0.076471
5,qualidade_servico,9,0.052941
7,qualidade_produto,9,0.052941
9,nivel_uso,9,0.052941
4,imagem_fornecedor,7,0.041176
10,nivel_satisfacao,7,0.041176
6,imagem_forca_vendas,6,0.035294
8,tamanho_firma,2,0.011765
